# PupilCheck - Pupil Segmentation Model Training

Two-model approach for in-browser and cloud pupil/iris detection:
1. **Pupil ROI Model** (~1-2MB TFLite) - Runs in-browser via TensorFlow.js
2. **Full Image Model** (~10-15MB SavedModel) - Runs on Cloud Run

**Real Datasets:**
- MOBIUS (3,559 images with masks, RGB phone cameras)
- iBUG Eye Segmentation (~2K images)
- Roboflow pupilX (804 images)
- Enhanced synthetic augmentation (15,000 images)

**Total: ~20,000+ training samples**

In [ ]:
# Install dependencies (local Jupyter)
!pip install -q tensorflow opencv-python-headless albumentations pillow scikit-learn matplotlib requests tqdm
# Optional: pip install roboflow  (for Roboflow API download)

In [ ]:
import os, json, glob, time, shutil, requests
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from pathlib import Path
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
import albumentations as A
from sklearn.model_selection import train_test_split
import matplotlib
matplotlib.use('Agg')  # headless rendering (no display needed)
import matplotlib.pyplot as plt
from tqdm import tqdm

# ---- Quick mode flag ----
# Set QUICK_MODE = True for fast validation (~5 min on CPU)
# Set QUICK_MODE = False for full training (~45-90 min on GPU)
QUICK_MODE = os.environ.get('QUICK_MODE', 'false').lower() == 'true'

# Configuration
DATA_DIR = Path('./data')      # Relative to notebook
MODELS_DIR = Path('./models')  # Export directory
IMG_SIZE_FULL = 256            # Full image model input
IMG_SIZE_ROI = 128             # ROI model input
NUM_CLASSES = 3                # bg, iris, pupil
BATCH_SIZE = 16 if not QUICK_MODE else 8

for d in [DATA_DIR, MODELS_DIR,
          DATA_DIR / 'mobius', DATA_DIR / 'ibug',
          DATA_DIR / 'roboflow', DATA_DIR / 'synthetic']:
    d.mkdir(parents=True, exist_ok=True)

print(f"TensorFlow: {tf.__version__}")
print(f"GPU: {tf.config.list_physical_devices('GPU')}")
print(f"Quick mode: {QUICK_MODE}")
if QUICK_MODE:
    print("  (500 synthetic samples, 3+3 epochs -- pipeline validation only)")

## Dataset Download Instructions

Place datasets in the `data/` directory next to this notebook.  
The notebook works with **synthetic data only** if no real datasets are present -- you can train with just synthetic data and add real data later for improved accuracy.

### MOBIUS (3,559 images, RGB phone cameras) -- RECOMMENDED
1. Visit https://sclera.fri.uni-lj.si/datasets.html
2. Fill out the download request form (requires academic/institutional email)
3. You will receive a download link via email
4. Extract into `data/mobius/`
Expected structure: `data/mobius/` with parallel `images/` and `masks/` subdirectories (or any nested structure with matching image-mask pairs -- the loader auto-discovers them).

### iBUG Eye Segmentation (~8,882 eye patches)
1. Visit https://ibug.doc.ic.ac.uk/resources/ibug-eye-segmentation-dataset/
2. Follow the download/request instructions on the page
3. Extract into `data/ibug/`
Expected structure: `data/ibug/images/` and `data/ibug/masks/`

### Roboflow pupilX (804 images, CC BY 4.0)
**Option A -- Roboflow Python SDK (recommended):**
```bash
pip install roboflow
```
```python
from roboflow import Roboflow
rf = Roboflow(api_key="YOUR_API_KEY")  # Free signup at https://app.roboflow.com
project = rf.workspace("pupil-exnzt").project("pupilx")
dataset = project.version(1).download("png-mask-semantic", location="data/roboflow")
```
**Option B -- Manual download:**
1. Visit https://universe.roboflow.com/pupil-exnzt/pupilx
2. Click "Download Dataset" and choose "Semantic Segmentation" format
3. Extract into `data/roboflow/`
Expected structure: `data/roboflow/train/`, `data/roboflow/valid/`, `data/roboflow/test/` each containing `images/` and `masks/` subdirs.

### No datasets? No problem!
If none of the real datasets are available, the notebook will automatically generate 15,000 enhanced synthetic eye images and train on those. You can always add real data later and retrain for better accuracy.

In [ ]:
# ============================================================
# Shared utilities for all dataset loaders
# ============================================================

def find_mask_path(img_path):
    """Find the mask file corresponding to an image using common naming conventions."""
    img_path = str(img_path)
    base = os.path.basename(img_path)
    name_no_ext = os.path.splitext(base)[0]
    parent = os.path.dirname(img_path)
    grandparent = os.path.dirname(parent)

    # Pattern 1: parallel directory (images/ -> masks/, labels/, annotations/)
    for src_dir in ['images', 'image', 'imgs', 'img']:
        if f'/{src_dir}/' in img_path or f'/{src_dir}' == img_path[-len(src_dir)-1:]:
            for dst_dir in ['masks', 'mask', 'labels', 'label', 'annotations', 'segs']:
                candidate = img_path.replace(f'/{src_dir}/', f'/{dst_dir}/')
                for ext in ['.png', '.jpg', '.bmp', '.tif']:
                    c = os.path.splitext(candidate)[0] + ext
                    if os.path.exists(c):
                        return c

    # Pattern 2: same directory, mask suffix
    for suffix in ['_mask', '_label', '_seg', '_gt', '.mask', '.label']:
        for ext in ['.png', '.jpg', '.bmp']:
            candidate = os.path.join(parent, name_no_ext + suffix + ext)
            if os.path.exists(candidate):
                return candidate

    # Pattern 3: sibling directory from grandparent
    for subdir in ['masks', 'labels', 'annotations', 'groundtruth', 'gt']:
        for ext in ['.png', '.jpg', '.bmp', '.tif']:
            candidate = os.path.join(grandparent, subdir, name_no_ext + ext)
            if os.path.exists(candidate):
                return candidate

    return None


def remap_mask(mask):
    """Remap dataset-specific mask values to {0: bg, 1: iris, 2: pupil}."""
    unique = np.unique(mask)
    new_mask = np.zeros_like(mask)

    if len(unique) <= 4:
        # Assume ascending order: bg=0, (sclera), iris, pupil
        if len(unique) == 3:  # bg, iris, pupil
            new_mask[mask == unique[1]] = 1  # iris
            new_mask[mask == unique[2]] = 2  # pupil
        elif len(unique) == 4:  # bg, sclera, iris, pupil
            new_mask[mask == unique[2]] = 1  # iris
            new_mask[mask == unique[3]] = 2  # pupil
        elif len(unique) == 2:  # bg + one class (assume pupil)
            new_mask[mask == unique[1]] = 2
    else:
        # Intensity-coded: dark=pupil, mid=iris, bright=bg/sclera
        new_mask[mask > 200] = 0   # bright = background/sclera
        new_mask[(mask > 50) & (mask <= 200)] = 1  # mid = iris
        new_mask[mask <= 50] = 2   # dark = pupil

    return new_mask


def extract_iris_roi(image, mask, roi_size):
    """Extract iris ROI crop for the pupil ROI model.

    Returns:
        roi_img: (roi_size, roi_size, 3) uint8 or None
        roi_msk: (roi_size, roi_size) binary uint8 (1=pupil) or None
    """
    iris_mask = (mask == 1) | (mask == 2)
    if not np.any(iris_mask):
        return None, None

    ys, xs = np.where(iris_mask)
    cx, cy = int(np.mean(xs)), int(np.mean(ys))

    # Estimate iris radius from mask extents
    dists = np.sqrt((xs - cx) ** 2 + (ys - cy) ** 2)
    ir = int(np.percentile(dists, 90))

    # Crop with padding
    pad = int(ir * 1.3)
    x1 = max(0, cx - pad)
    y1 = max(0, cy - pad)
    x2 = min(image.shape[1], cx + pad)
    y2 = min(image.shape[0], cy + pad)

    roi_img = image[y1:y2, x1:x2]
    roi_msk = mask[y1:y2, x1:x2]

    if roi_img.shape[0] < 10 or roi_img.shape[1] < 10:
        return None, None

    roi_img = cv2.resize(roi_img, (roi_size, roi_size))
    roi_msk = cv2.resize(roi_msk, (roi_size, roi_size), interpolation=cv2.INTER_NEAREST)

    # Binary pupil mask for ROI model
    roi_msk_binary = (roi_msk == 2).astype(np.uint8)
    return roi_img, roi_msk_binary


print("Shared utilities loaded.")

In [ ]:
# ============================================================
# MOBIUS Dataset Loader
# ============================================================

def load_mobius_dataset():
    """Load MOBIUS dataset (phone camera eye images with segmentation masks).

    Returns:
        full_images:  list of (256,256,3) uint8
        full_masks:   list of (256,256) uint8  {0,1,2}
        roi_pairs:    list of (roi_img, roi_msk_binary) tuples
    """
    mobius_dir = DATA_DIR / 'mobius'
    if not mobius_dir.exists():
        print("MOBIUS directory not found. Skipping.")
        return [], [], []

    # Collect all image files recursively
    img_files = sorted(
        glob.glob(str(mobius_dir / '**' / '*.png'), recursive=True) +
        glob.glob(str(mobius_dir / '**' / '*.jpg'), recursive=True) +
        glob.glob(str(mobius_dir / '**' / '*.bmp'), recursive=True)
    )

    # Filter out mask/label files
    img_files = [
        f for f in img_files
        if not any(kw in f.lower() for kw in ['mask', 'label', 'seg', 'gt', 'annot'])
    ]

    print(f"Found {len(img_files)} MOBIUS candidate image files")

    full_images, full_masks, roi_pairs = [], [], []

    for img_path in tqdm(img_files, desc="Loading MOBIUS"):
        mask_path = find_mask_path(img_path)
        if not mask_path or not os.path.exists(mask_path):
            continue

        img = cv2.imread(img_path)
        msk = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if img is None or msk is None:
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Resize for full model
        img_full = cv2.resize(img, (IMG_SIZE_FULL, IMG_SIZE_FULL))
        msk_full = cv2.resize(msk, (IMG_SIZE_FULL, IMG_SIZE_FULL),
                              interpolation=cv2.INTER_NEAREST)
        msk_full = remap_mask(msk_full)

        full_images.append(img_full)
        full_masks.append(msk_full)

        # Extract iris ROI for ROI model
        roi_img, roi_msk = extract_iris_roi(img_full, msk_full, IMG_SIZE_ROI)
        if roi_img is not None:
            roi_pairs.append((roi_img, roi_msk))

    print(f"Loaded {len(full_images)} MOBIUS samples "
          f"({len(roi_pairs)} ROI crops)")
    return full_images, full_masks, roi_pairs


mobius_imgs, mobius_masks, mobius_rois = load_mobius_dataset()

In [ ]:
# ============================================================
# iBUG Eye Segmentation Loader
# ============================================================

def load_ibug_dataset():
    """Load iBUG sclera/eye segmentation dataset.

    The iBUG dataset provides sclera segmentation masks.
    We derive iris and pupil regions from the non-sclera eye area.

    Returns:
        full_images, full_masks, roi_pairs
    """
    ibug_dir = DATA_DIR / 'ibug'
    if not ibug_dir.exists():
        print("iBUG directory not found. Skipping.")
        return [], [], []

    img_files = sorted(
        glob.glob(str(ibug_dir / '**' / '*.png'), recursive=True) +
        glob.glob(str(ibug_dir / '**' / '*.jpg'), recursive=True) +
        glob.glob(str(ibug_dir / '**' / '*.bmp'), recursive=True)
    )

    img_files = [
        f for f in img_files
        if not any(kw in f.lower() for kw in ['mask', 'label', 'seg', 'gt', 'annot'])
    ]

    print(f"Found {len(img_files)} iBUG candidate image files")

    full_images, full_masks, roi_pairs = [], [], []

    for img_path in tqdm(img_files, desc="Loading iBUG"):
        mask_path = find_mask_path(img_path)
        if not mask_path or not os.path.exists(mask_path):
            continue

        img = cv2.imread(img_path)
        msk = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if img is None or msk is None:
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        img_full = cv2.resize(img, (IMG_SIZE_FULL, IMG_SIZE_FULL))
        msk_full = cv2.resize(msk, (IMG_SIZE_FULL, IMG_SIZE_FULL),
                              interpolation=cv2.INTER_NEAREST)

        # iBUG provides sclera masks - remap to our classes
        # Common encoding: sclera=white(255), iris/pupil=other values
        msk_full = remap_mask(msk_full)

        full_images.append(img_full)
        full_masks.append(msk_full)

        roi_img, roi_msk = extract_iris_roi(img_full, msk_full, IMG_SIZE_ROI)
        if roi_img is not None:
            roi_pairs.append((roi_img, roi_msk))

    print(f"Loaded {len(full_images)} iBUG samples "
          f"({len(roi_pairs)} ROI crops)")
    return full_images, full_masks, roi_pairs


ibug_imgs, ibug_masks, ibug_rois = load_ibug_dataset()

In [ ]:
# ============================================================
# Roboflow pupilX Loader
# ============================================================

def load_roboflow_dataset():
    """Load Roboflow pupilX dataset (iris + pupil instance segmentation).

    Supports both API-downloaded and manually-placed data.
    Expected directory structure:
        data/roboflow/
            train/images/  train/masks/
            valid/images/  valid/masks/
            test/images/   test/masks/
    OR flat:
        data/roboflow/images/  data/roboflow/masks/

    Returns:
        full_images, full_masks, roi_pairs
    """
    rb_dir = DATA_DIR / 'roboflow'
    if not rb_dir.exists():
        print("Roboflow directory not found. Skipping.")
        return [], [], []

    # Attempt API download if roboflow package is available and no images exist
    existing = glob.glob(str(rb_dir / '**' / '*.jpg'), recursive=True) + \
               glob.glob(str(rb_dir / '**' / '*.png'), recursive=True)

    if len(existing) == 0:
        try:
            from roboflow import Roboflow
            api_key = os.environ.get('ROBOFLOW_API_KEY', '')
            if api_key:
                print("Downloading pupilX dataset via Roboflow API...")
                rf = Roboflow(api_key=api_key)
                project = rf.workspace().project("pupilx")
                dataset = project.version(1).download(
                    "png-mask-semantic", location=str(rb_dir)
                )
                print("Roboflow download complete.")
            else:
                print("Set ROBOFLOW_API_KEY env var to auto-download, "
                      "or place data manually in data/roboflow/")
                return [], [], []
        except ImportError:
            print("roboflow package not installed. "
                  "Place data manually in data/roboflow/")
            return [], [], []

    # Collect images from all splits
    img_files = sorted(
        glob.glob(str(rb_dir / '**' / 'images' / '*.jpg'), recursive=True) +
        glob.glob(str(rb_dir / '**' / 'images' / '*.png'), recursive=True)
    )

    if not img_files:
        # Try flat layout
        all_files = sorted(
            glob.glob(str(rb_dir / '**' / '*.jpg'), recursive=True) +
            glob.glob(str(rb_dir / '**' / '*.png'), recursive=True)
        )
        img_files = [
            f for f in all_files
            if not any(kw in f.lower() for kw in ['mask', 'label', 'seg'])
        ]

    print(f"Found {len(img_files)} Roboflow candidate image files")

    full_images, full_masks, roi_pairs = [], [], []

    for img_path in tqdm(img_files, desc="Loading Roboflow"):
        mask_path = find_mask_path(img_path)
        if not mask_path or not os.path.exists(mask_path):
            continue

        img = cv2.imread(img_path)
        msk = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if img is None or msk is None:
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        img_full = cv2.resize(img, (IMG_SIZE_FULL, IMG_SIZE_FULL))
        msk_full = cv2.resize(msk, (IMG_SIZE_FULL, IMG_SIZE_FULL),
                              interpolation=cv2.INTER_NEAREST)
        msk_full = remap_mask(msk_full)

        full_images.append(img_full)
        full_masks.append(msk_full)

        roi_img, roi_msk = extract_iris_roi(img_full, msk_full, IMG_SIZE_ROI)
        if roi_img is not None:
            roi_pairs.append((roi_img, roi_msk))

    print(f"Loaded {len(full_images)} Roboflow samples "
          f"({len(roi_pairs)} ROI crops)")
    return full_images, full_masks, roi_pairs


roboflow_imgs, roboflow_masks, roboflow_rois = load_roboflow_dataset()

## Enhanced Synthetic Data Generation

Generates realistic synthetic eye images with:
- Curved eyelid occlusion (Bezier-style)
- Specular highlights (Purkinje images)
- Color cast and white-balance variations
- Motion blur and defocus
- Realistic iris textures with radial fibers
- Off-center pupils for anisocoria simulation

Each sample produces **both** a full-image (256x256, 3-class) mask
and an iris-ROI crop (128x128, binary pupil mask).

In [ ]:
def generate_synthetic_eye(img_size=256, pupil_ratio=None, anisocoria=False):
    """Generate a synthetic eye image with precise segmentation mask.

    Returns:
        image: RGB uint8 array (img_size, img_size, 3)
        mask:  uint8 array (img_size, img_size) with {0:bg, 1:iris, 2:pupil}
    """
    img = np.zeros((img_size, img_size, 3), dtype=np.uint8)
    mask = np.zeros((img_size, img_size), dtype=np.uint8)

    center_x = img_size // 2 + np.random.randint(-15, 16)
    center_y = img_size // 2 + np.random.randint(-15, 16)
    center = (center_x, center_y)

    # --- Iris parameters ---
    iris_radius = np.random.randint(img_size // 4, img_size // 3)

    iris_colors = [
        (np.random.randint(40, 90),
         np.random.randint(60, 120),
         np.random.randint(100, 180)),  # brown
        (np.random.randint(140, 200),
         np.random.randint(100, 160),
         np.random.randint(40, 80)),    # blue
        (np.random.randint(60, 120),
         np.random.randint(120, 180),
         np.random.randint(60, 100)),   # green
        (np.random.randint(80, 130),
         np.random.randint(100, 150),
         np.random.randint(120, 180)),  # hazel
        (np.random.randint(120, 160),
         np.random.randint(120, 160),
         np.random.randint(120, 160)),  # gray
    ]
    iris_color = iris_colors[np.random.randint(0, len(iris_colors))]

    # --- Pupil parameters ---
    if pupil_ratio is None:
        if anisocoria:
            pupil_ratio = np.random.choice([
                np.random.uniform(0.15, 0.25),  # very constricted
                np.random.uniform(0.60, 0.80),  # very dilated
            ])
        else:
            pupil_ratio = np.random.uniform(0.25, 0.65)

    pupil_radius = int(iris_radius * pupil_ratio)

    # Off-center pupil (more pronounced for anisocoria)
    off_range = 6 if anisocoria else 3
    pupil_offset_x = np.random.randint(-off_range, off_range + 1)
    pupil_offset_y = np.random.randint(-off_range, off_range + 1)
    pupil_center = (center_x + pupil_offset_x, center_y + pupil_offset_y)

    # --- Sclera ---
    sclera_color = (
        np.random.randint(220, 250),
        np.random.randint(220, 250),
        np.random.randint(225, 255),
    )
    sclera_w = int(iris_radius * np.random.uniform(1.6, 2.2))
    sclera_h = int(iris_radius * np.random.uniform(1.1, 1.4))
    cv2.ellipse(img, center, (sclera_w, sclera_h), 0, 0, 360, sclera_color, -1)

    # --- Iris with radial gradient + limbal ring ---
    for r in range(iris_radius, 0, -1):
        t = r / iris_radius
        darken = 0.4 + 0.6 * t  # darker limbal ring
        c = tuple(int(v * darken) for v in iris_color)
        cv2.circle(img, center, r, c, -1)
    cv2.circle(mask, center, iris_radius, 1, -1)

    # --- Iris texture (radial fibers + collarette) ---
    num_fibers = np.random.randint(30, 60)
    for _ in range(num_fibers):
        angle = np.random.uniform(0, 2 * np.pi)
        r_start = pupil_radius * np.random.uniform(1.05, 1.2)
        r_end = iris_radius * np.random.uniform(0.5, 0.95)
        x1 = int(center_x + np.cos(angle) * r_start)
        y1 = int(center_y + np.sin(angle) * r_start)
        x2 = int(center_x + np.cos(angle) * r_end)
        y2 = int(center_y + np.sin(angle) * r_end)
        fc = tuple(max(0, min(255, int(v * np.random.uniform(0.6, 1.4))))
                   for v in iris_color)
        cv2.line(img, (x1, y1), (x2, y2), fc, 1)

    # Collarette ring (mid-iris ring)
    collarette_r = int(iris_radius * np.random.uniform(0.45, 0.55))
    col_color = tuple(max(0, min(255, int(v * 1.2))) for v in iris_color)
    cv2.circle(img, center, collarette_r, col_color, 1)

    # --- Pupil ---
    pupil_darkness = np.random.randint(3, 25)
    cv2.circle(img, pupil_center, pupil_radius,
               (pupil_darkness, pupil_darkness, pupil_darkness), -1)
    cv2.circle(mask, pupil_center, pupil_radius, 2, -1)

    # --- Specular highlights (Purkinje images) ---
    num_reflections = np.random.randint(1, 4)
    for _ in range(num_reflections):
        ref_x = center_x + np.random.randint(-iris_radius // 2, iris_radius // 2)
        ref_y = center_y + np.random.randint(-iris_radius // 3, iris_radius // 4)
        ref_r = np.random.randint(2, max(3, int(iris_radius * 0.08)))
        brightness = np.random.randint(220, 256)
        cv2.circle(img, (ref_x, ref_y), ref_r,
                   (brightness, brightness, min(255, brightness + 10)), -1)
        # Soft glow around highlight
        cv2.circle(img, (ref_x, ref_y), ref_r + 2,
                   (brightness // 2, brightness // 2, brightness // 2), 1)

    # --- Curved eyelids (Bezier-style using polylines) ---
    if np.random.random() > 0.2:
        skin_color = (
            np.random.randint(140, 220),
            np.random.randint(120, 200),
            np.random.randint(100, 180),
        )

        # Top eyelid - curved
        lid_drop_top = np.random.uniform(0.6, 1.3) * iris_radius
        lid_top_y = int(center_y - lid_drop_top)
        lid_curve_top = np.random.randint(10, 35)
        n_pts = 30
        xs_top = np.linspace(0, img_size, n_pts).astype(int)
        # Parabolic curve for upper lid
        ys_top = (lid_top_y
                  - lid_curve_top
                  * np.sin(np.linspace(0, np.pi, n_pts))).astype(int)
        pts_top = np.column_stack([xs_top, ys_top])
        pts_top = np.vstack([[img_size, 0], [0, 0], pts_top])
        cv2.fillPoly(img, [pts_top.astype(np.int32)], skin_color)
        cv2.fillPoly(mask, [pts_top.astype(np.int32)], 0)
        # Eyelash line
        for k in range(len(xs_top) - 1):
            lash_color = tuple(max(0, c - 60) for c in skin_color)
            cv2.line(img, (xs_top[k], ys_top[k]),
                     (xs_top[k + 1], ys_top[k + 1]), lash_color, 2)

        # Bottom eyelid - curved
        lid_drop_bot = np.random.uniform(0.6, 1.3) * iris_radius
        lid_bot_y = int(center_y + lid_drop_bot)
        lid_curve_bot = np.random.randint(5, 20)
        xs_bot = np.linspace(0, img_size, n_pts).astype(int)
        ys_bot = (lid_bot_y
                  + lid_curve_bot
                  * np.sin(np.linspace(0, np.pi, n_pts))).astype(int)
        pts_bot = np.column_stack([xs_bot, ys_bot])
        pts_bot = np.vstack([pts_bot, [img_size, img_size], [0, img_size]])
        cv2.fillPoly(img, [pts_bot.astype(np.int32)], skin_color)
        cv2.fillPoly(mask, [pts_bot.astype(np.int32)], 0)

    # --- Color cast / white-balance variation ---
    if np.random.random() > 0.5:
        cast = np.array([
            np.random.uniform(-15, 15),
            np.random.uniform(-10, 10),
            np.random.uniform(-15, 15),
        ])
        img = np.clip(img.astype(np.float32) + cast, 0, 255).astype(np.uint8)

    # --- Noise ---
    noise_sigma = np.random.uniform(3, 15)
    noise = np.random.normal(0, noise_sigma, img.shape).astype(np.int16)
    img = np.clip(img.astype(np.int16) + noise, 0, 255).astype(np.uint8)

    # --- Random brightness / contrast ---
    alpha = np.random.uniform(0.7, 1.3)
    beta = np.random.randint(-30, 31)
    img = np.clip(alpha * img.astype(np.float32) + beta, 0, 255).astype(np.uint8)

    # --- Motion blur (occasional) ---
    if np.random.random() > 0.8:
        ksize = np.random.choice([3, 5, 7])
        kernel = np.zeros((ksize, ksize))
        if np.random.random() > 0.5:
            kernel[ksize // 2, :] = 1.0 / ksize   # horizontal
        else:
            kernel[:, ksize // 2] = 1.0 / ksize   # vertical
        img = cv2.filter2D(img, -1, kernel)

    return img, mask


# Preview synthetic samples
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
for i in range(4):
    aniso = i >= 2
    img, msk = generate_synthetic_eye(IMG_SIZE_FULL, anisocoria=aniso)
    axes[0, i].imshow(img)
    axes[0, i].set_title(f"{'Anisocoria' if aniso else 'Normal'} eye")
    axes[0, i].axis('off')
    axes[1, i].imshow(msk, cmap='viridis', vmin=0, vmax=2)
    axes[1, i].set_title('Mask (0=bg, 1=iris, 2=pupil)')
    axes[1, i].axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# Generate synthetic dataset
# ============================================================

if QUICK_MODE:
    NUM_NORMAL = 400
    NUM_ANISOCORIA = 100
else:
    NUM_NORMAL = 10000
    NUM_ANISOCORIA = 5000

print(f"Generating {NUM_NORMAL + NUM_ANISOCORIA} synthetic eye images...")

syn_full_images, syn_full_masks, syn_roi_pairs = [], [], []

# Normal pupils
for _ in tqdm(range(NUM_NORMAL), desc="Synthetic normal"):
    img, msk = generate_synthetic_eye(IMG_SIZE_FULL)
    syn_full_images.append(img)
    syn_full_masks.append(msk)
    roi_img, roi_msk = extract_iris_roi(img, msk, IMG_SIZE_ROI)
    if roi_img is not None:
        syn_roi_pairs.append((roi_img, roi_msk))

# Anisocoria cases
for _ in tqdm(range(NUM_ANISOCORIA), desc="Synthetic anisocoria"):
    img, msk = generate_synthetic_eye(IMG_SIZE_FULL, anisocoria=True)
    syn_full_images.append(img)
    syn_full_masks.append(msk)
    roi_img, roi_msk = extract_iris_roi(img, msk, IMG_SIZE_ROI)
    if roi_img is not None:
        syn_roi_pairs.append((roi_img, roi_msk))

print(f"\nSynthetic: {len(syn_full_images)} full images, "
      f"{len(syn_roi_pairs)} ROI crops")

In [ ]:
# ============================================================
# Merge all datasets and create train/val splits
# ============================================================

# --- Full image model data (256x256, 3-class) ---
all_full_images = (mobius_imgs + ibug_imgs + roboflow_imgs + syn_full_images)
all_full_masks  = (mobius_masks + ibug_masks + roboflow_masks + syn_full_masks)

# --- ROI model data (128x128, binary) ---
all_roi_pairs = (mobius_rois + ibug_rois + roboflow_rois + syn_roi_pairs)

print(f"Dataset summary:")
print(f"  MOBIUS    : {len(mobius_imgs):>6} full / {len(mobius_rois):>6} ROI")
print(f"  iBUG      : {len(ibug_imgs):>6} full / {len(ibug_rois):>6} ROI")
print(f"  Roboflow  : {len(roboflow_imgs):>6} full / {len(roboflow_rois):>6} ROI")
print(f"  Synthetic : {len(syn_full_images):>6} full / {len(syn_roi_pairs):>6} ROI")
print(f"  " + "-" * 40)
print(f"  TOTAL     : {len(all_full_images):>6} full / {len(all_roi_pairs):>6} ROI")

# Convert to numpy arrays
full_images = np.array(all_full_images, dtype=np.uint8)
full_masks  = np.array(all_full_masks, dtype=np.uint8)

roi_images = np.array([p[0] for p in all_roi_pairs], dtype=np.uint8)
roi_masks  = np.array([p[1] for p in all_roi_pairs], dtype=np.uint8)

# Train / validation split (85% / 15%)
X_full_train, X_full_val, y_full_train, y_full_val = train_test_split(
    full_images, full_masks, test_size=0.15, random_state=42
)

X_roi_train, X_roi_val, y_roi_train, y_roi_val = train_test_split(
    roi_images, roi_masks, test_size=0.15, random_state=42
)

print(f"\nFull model:  train={len(X_full_train)}, val={len(X_full_val)}")
print(f"ROI  model:  train={len(X_roi_train)}, val={len(X_roi_val)}")

# Free raw lists to save memory
del all_full_images, all_full_masks, all_roi_pairs
del mobius_imgs, mobius_masks, mobius_rois
del ibug_imgs, ibug_masks, ibug_rois
del roboflow_imgs, roboflow_masks, roboflow_rois
del syn_full_images, syn_full_masks, syn_roi_pairs

# Mask class distribution (full model)
print(f"\nFull mask class distribution (train):")
print(f"  bg   = {np.mean(y_full_train == 0):.1%}")
print(f"  iris = {np.mean(y_full_train == 1):.1%}")
print(f"  pupil= {np.mean(y_full_train == 2):.1%}")

## Augmentation Pipeline

Shared augmentation applied on-the-fly via `tf.data` pipelines.
Uses `albumentations` for photometric + geometric transforms.

In [ ]:
# ============================================================
# Augmentation definitions (albumentations 2.0+ API)
# ============================================================

augment_full = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Affine(
        scale=(0.85, 1.15), translate_percent=(-0.1, 0.1),
        rotate=(-15, 15), p=0.7, border_mode=cv2.BORDER_CONSTANT),
    A.OneOf([
        A.GaussianBlur(blur_limit=(3, 7), p=1),
        A.MotionBlur(blur_limit=(3, 7), p=1),
    ], p=0.3),
    A.OneOf([
        A.RandomBrightnessContrast(brightness_limit=0.3,
                                   contrast_limit=0.3, p=1),
        A.CLAHE(clip_limit=4.0, p=1),
    ], p=0.5),
    A.GaussNoise(p=0.3),
    A.ColorJitter(brightness=0.2, contrast=0.2,
                  saturation=0.2, hue=0.05, p=0.4),
    A.CoarseDropout(num_holes_range=(1, 3),
                    hole_height_range=(10, 30),
                    hole_width_range=(10, 30),
                    fill=0, p=0.2),
])

augment_roi = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Affine(
        scale=(0.88, 1.12), translate_percent=(-0.08, 0.08),
        rotate=(-10, 10), p=0.6, border_mode=cv2.BORDER_CONSTANT),
    A.OneOf([
        A.GaussianBlur(blur_limit=(3, 5), p=1),
        A.MotionBlur(blur_limit=(3, 5), p=1),
    ], p=0.25),
    A.RandomBrightnessContrast(brightness_limit=0.25,
                               contrast_limit=0.25, p=0.5),
    A.GaussNoise(p=0.3),
    A.ColorJitter(brightness=0.15, contrast=0.15,
                  saturation=0.15, hue=0.04, p=0.3),
])


# ============================================================
# tf.data pipeline builders
# ============================================================

def _make_full_dataset(images, masks, batch_size, aug_fn=None, shuffle=True):
    """Build tf.data pipeline for the 3-class full-image model."""
    def _augment(image, mask):
        def _apply(img, msk):
            img_np = (img.numpy() * 255).astype(np.uint8)
            msk_np = np.argmax(msk.numpy(), axis=-1).astype(np.uint8)
            result = aug_fn(image=img_np, mask=msk_np)
            aug_img = result['image'].astype(np.float32) / 255.0
            aug_msk = tf.keras.utils.to_categorical(
                result['mask'], NUM_CLASSES).astype(np.float32)
            return aug_img, aug_msk

        img_a, msk_a = tf.py_function(
            _apply, [image, mask], [tf.float32, tf.float32])
        img_a.set_shape(image.shape)
        msk_a.set_shape(mask.shape)
        return img_a, msk_a

    imgs_f = images.astype(np.float32) / 255.0
    msks_oh = tf.keras.utils.to_categorical(masks, NUM_CLASSES).astype(np.float32)

    ds = tf.data.Dataset.from_tensor_slices((imgs_f, msks_oh))
    if shuffle:
        ds = ds.shuffle(buffer_size=min(len(images), 4000))
    if aug_fn is not None:
        ds = ds.map(_augment, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds


def _make_roi_dataset(images, masks, batch_size, aug_fn=None, shuffle=True):
    """Build tf.data pipeline for the binary ROI model."""
    def _augment(image, mask):
        def _apply(img, msk):
            img_np = (img.numpy() * 255).astype(np.uint8)
            msk_np = (msk.numpy()[..., 0] * 255).astype(np.uint8)
            result = aug_fn(image=img_np, mask=msk_np)
            aug_img = result['image'].astype(np.float32) / 255.0
            aug_msk = (result['mask'].astype(np.float32) / 255.0)[..., np.newaxis]
            return aug_img, aug_msk

        img_a, msk_a = tf.py_function(
            _apply, [image, mask], [tf.float32, tf.float32])
        img_a.set_shape(image.shape)
        msk_a.set_shape(mask.shape)
        return img_a, msk_a

    imgs_f = images.astype(np.float32) / 255.0
    msks_f = masks.astype(np.float32)[..., np.newaxis]  # (N, H, W, 1)

    ds = tf.data.Dataset.from_tensor_slices((imgs_f, msks_f))
    if shuffle:
        ds = ds.shuffle(buffer_size=min(len(images), 4000))
    if aug_fn is not None:
        ds = ds.map(_augment, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds


# Build datasets
train_full_ds = _make_full_dataset(X_full_train, y_full_train,
                                   BATCH_SIZE, aug_fn=augment_full)
val_full_ds   = _make_full_dataset(X_full_val, y_full_val,
                                   BATCH_SIZE, aug_fn=None, shuffle=False)

train_roi_ds = _make_roi_dataset(X_roi_train, y_roi_train,
                                 BATCH_SIZE, aug_fn=augment_roi)
val_roi_ds   = _make_roi_dataset(X_roi_val, y_roi_val,
                                 BATCH_SIZE, aug_fn=None, shuffle=False)

print(f"Full  train batches: {tf.data.experimental.cardinality(train_full_ds).numpy()}")
print(f"Full  val   batches: {tf.data.experimental.cardinality(val_full_ds).numpy()}")
print(f"ROI   train batches: {tf.data.experimental.cardinality(train_roi_ds).numpy()}")
print(f"ROI   val   batches: {tf.data.experimental.cardinality(val_roi_ds).numpy()}")

## Model A -- Pupil ROI Model

Tiny U-Net for **binary pupil segmentation** within an iris ROI crop.  
- Input: 128x128x3 (cropped iris region)  
- Output: 128x128x1 sigmoid (pupil probability)  
- **No pretrained backbone** -- keeps the TFLite export under 2 MB  
- Designed for in-browser inference via TensorFlow.js

In [ ]:
def build_pupil_roi_model(input_shape=(128, 128, 3)):
    """Tiny U-Net for binary pupil segmentation within iris ROI.
    Target: <2 MB TFLite, runs in-browser via TF.js.
    """
    inputs = layers.Input(shape=input_shape)

    # --- Encoder ---
    c1 = layers.Conv2D(16, 3, padding='same', activation='relu')(inputs)
    c1 = layers.Conv2D(16, 3, padding='same', activation='relu')(c1)
    p1 = layers.MaxPooling2D()(c1)  # 64x64

    c2 = layers.Conv2D(32, 3, padding='same', activation='relu')(p1)
    c2 = layers.Conv2D(32, 3, padding='same', activation='relu')(c2)
    p2 = layers.MaxPooling2D()(c2)  # 32x32

    c3 = layers.Conv2D(64, 3, padding='same', activation='relu')(p2)
    c3 = layers.Conv2D(64, 3, padding='same', activation='relu')(c3)
    p3 = layers.MaxPooling2D()(c3)  # 16x16

    # --- Bottleneck ---
    c4 = layers.Conv2D(128, 3, padding='same', activation='relu')(p3)
    c4 = layers.Conv2D(128, 3, padding='same', activation='relu')(c4)

    # --- Decoder ---
    u3 = layers.UpSampling2D()(c4)  # 32x32
    u3 = layers.Concatenate()([u3, c3])
    u3 = layers.Conv2D(64, 3, padding='same', activation='relu')(u3)

    u2 = layers.UpSampling2D()(u3)  # 64x64
    u2 = layers.Concatenate()([u2, c2])
    u2 = layers.Conv2D(32, 3, padding='same', activation='relu')(u2)

    u1 = layers.UpSampling2D()(u2)  # 128x128
    u1 = layers.Concatenate()([u1, c1])
    u1 = layers.Conv2D(16, 3, padding='same', activation='relu')(u1)

    # --- Output: single channel sigmoid (binary mask) ---
    outputs = layers.Conv2D(1, 1, activation='sigmoid',
                            name='pupil_mask')(u1)

    model = Model(inputs, outputs, name='pupil_roi_segnet')
    return model


roi_model = build_pupil_roi_model()
roi_model.summary()

# Verify parameter count is small enough for <2 MB TFLite
total_params = roi_model.count_params()
est_size_mb = total_params * 2 / (1024 * 1024)  # float16
print(f"\nEstimated TFLite (float16) size: ~{est_size_mb:.1f} MB")

In [ ]:
# ============================================================
# Train Pupil ROI Model -- two-phase training
# ============================================================

# --- Loss functions for binary segmentation ---

def binary_dice_coeff(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.cast(tf.reshape(y_true, [-1]), tf.float32)
    y_pred_f = tf.cast(tf.reshape(y_pred, [-1]), tf.float32)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2.0 * intersection + smooth) / (
        tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)


def binary_dice_loss(y_true, y_pred):
    return 1.0 - binary_dice_coeff(y_true, y_pred)


def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    bce = tf.reduce_mean(bce)
    return bce + binary_dice_loss(y_true, y_pred)


def binary_iou(y_true, y_pred, threshold=0.5):
    y_pred_b = tf.cast(y_pred > threshold, tf.float32)
    y_true_f = tf.cast(tf.reshape(y_true, [-1]), tf.float32)
    y_pred_f = tf.reshape(y_pred_b, [-1])
    inter = tf.reduce_sum(y_true_f * y_pred_f)
    union = tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) - inter
    return (inter + 1e-6) / (union + 1e-6)


ROI_EPOCHS_P1 = 3 if QUICK_MODE else 15
ROI_EPOCHS_P2 = 3 if QUICK_MODE else 15

# --- Phase 1: full training (no frozen layers -- no pretrained backbone) ---
print("=" * 60)
print(f"ROI Model -- Phase 1: initial training (lr=1e-3, {ROI_EPOCHS_P1} epochs)")
print("=" * 60)

roi_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=bce_dice_loss,
    metrics=[binary_dice_coeff, binary_iou],
)

roi_callbacks_p1 = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_binary_dice_coeff', patience=5,
        mode='max', restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_binary_dice_coeff', factor=0.5,
        patience=3, mode='max', min_lr=1e-5),
]

roi_hist_p1 = roi_model.fit(
    train_roi_ds,
    validation_data=val_roi_ds,
    epochs=ROI_EPOCHS_P1,
    callbacks=roi_callbacks_p1,
)

# --- Phase 2: fine-tune with lower LR ---
print("\n" + "=" * 60)
print(f"ROI Model -- Phase 2: fine-tune (lr=1e-4, {ROI_EPOCHS_P2} epochs)")
print("=" * 60)

roi_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=bce_dice_loss,
    metrics=[binary_dice_coeff, binary_iou],
)

roi_callbacks_p2 = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_binary_dice_coeff', patience=7,
        mode='max', restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_binary_dice_coeff', factor=0.5,
        patience=3, mode='max', min_lr=1e-6),
    tf.keras.callbacks.ModelCheckpoint(
        str(MODELS_DIR / 'roi_best.keras'),
        monitor='val_binary_dice_coeff',
        mode='max', save_best_only=True),
]

roi_hist_p2 = roi_model.fit(
    train_roi_ds,
    validation_data=val_roi_ds,
    epochs=ROI_EPOCHS_P2,
    callbacks=roi_callbacks_p2,
)

## Model B -- Full Image Model

MobileNetV3-Small U-Net for **3-class segmentation** of the full eye image.  
- Input: 256x256x3 (full eye image)  
- Output: 256x256x3 softmax (background, iris, pupil)  
- Encoder: MobileNetV3-Small with ImageNet weights  
- Target: ~10-15 MB SavedModel, runs on Cloud Run

In [ ]:
def conv_block(x, filters, kernel_size=3):
    """Conv + BatchNorm + ReLU."""
    x = layers.Conv2D(filters, kernel_size, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x


def build_full_image_model(input_shape=(256, 256, 3), num_classes=3):
    """U-Net with MobileNetV3-Small encoder (ImageNet weights)."""

    base_model = tf.keras.applications.MobileNetV3Small(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet',
        minimalistic=True,
    )

    # Build a test input to get output shapes (Keras 3 compatible)
    test_input = tf.keras.Input(shape=input_shape)
    _ = base_model(test_input)

    # Find skip-connection layers at various spatial resolutions
    target_sizes = [128, 64, 32, 16, 8]
    skip_layers = []
    for target in target_sizes:
        for layer in reversed(base_model.layers):
            try:
                out_shape = layer.output.shape
            except (AttributeError, RuntimeError):
                continue
            if len(out_shape) == 4 and out_shape[1] == target:
                if layer.name not in skip_layers:
                    skip_layers.append(layer.name)
                    break

    print(f"Skip connection layers: {skip_layers}")

    encoder_outputs = [base_model.get_layer(n).output for n in skip_layers]
    encoder = Model(inputs=base_model.input, outputs=encoder_outputs)

    # Freeze encoder for phase 1
    encoder.trainable = False

    # Decoder
    inputs = layers.Input(shape=input_shape)
    skips = encoder(inputs)

    # Handle case where encoder has single output (not a list)
    if not isinstance(skips, list):
        skips = [skips]

    x = skips[-1]  # deepest features

    decoder_filters = [128, 64, 48, 32, 24]
    for i in range(len(skips) - 2, -1, -1):
        x = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(x)
        skip = skips[i]
        if x.shape[1] != skip.shape[1] or x.shape[2] != skip.shape[2]:
            x = layers.Resizing(skip.shape[1], skip.shape[2])(x)
        x = layers.Concatenate()([x, skip])
        f = decoder_filters[min(i, len(decoder_filters) - 1)]
        x = conv_block(x, f)
        x = conv_block(x, f)

    # Upsample to input resolution
    x = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(x)
    if x.shape[1] != input_shape[0]:
        x = layers.Resizing(input_shape[0], input_shape[1])(x)
    x = conv_block(x, 16)

    outputs = layers.Conv2D(num_classes, 1, activation='softmax',
                            name='segmentation')(x)

    model = Model(inputs=inputs, outputs=outputs,
                  name='full_image_segnet')
    return model, encoder


full_model, full_encoder = build_full_image_model()
full_model.summary()

In [ ]:
# ============================================================
# Train Full Image Model -- two-phase training
# ============================================================

# --- Loss functions for multi-class segmentation ---

def dice_coefficient(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.cast(tf.reshape(y_true, [-1, NUM_CLASSES]), tf.float32)
    y_pred_f = tf.cast(tf.reshape(y_pred, [-1, NUM_CLASSES]), tf.float32)
    intersection = tf.reduce_sum(y_true_f * y_pred_f, axis=0)
    union = tf.reduce_sum(y_true_f, axis=0) + tf.reduce_sum(y_pred_f, axis=0)
    dice = (2.0 * intersection + smooth) / (union + smooth)
    return tf.reduce_mean(dice[1:])  # exclude background


def dice_loss(y_true, y_pred):
    return 1.0 - dice_coefficient(y_true, y_pred)


def combined_loss(y_true, y_pred):
    ce = tf.reduce_mean(
        tf.keras.losses.categorical_crossentropy(y_true, y_pred))
    return ce + dice_loss(y_true, y_pred)


def pupil_iou(y_true, y_pred):
    y_true_p = y_true[..., 2]
    y_pred_p = tf.cast(tf.argmax(y_pred, axis=-1) == 2, tf.float32)
    inter = tf.reduce_sum(y_true_p * y_pred_p)
    union = tf.reduce_sum(y_true_p) + tf.reduce_sum(y_pred_p) - inter
    return (inter + 1e-6) / (union + 1e-6)


def iris_iou(y_true, y_pred):
    y_true_i = y_true[..., 1]
    y_pred_i = tf.cast(tf.argmax(y_pred, axis=-1) == 1, tf.float32)
    inter = tf.reduce_sum(y_true_i * y_pred_i)
    union = tf.reduce_sum(y_true_i) + tf.reduce_sum(y_pred_i) - inter
    return (inter + 1e-6) / (union + 1e-6)


FULL_EPOCHS_P1 = 3 if QUICK_MODE else 10
FULL_EPOCHS_P2 = 3 if QUICK_MODE else 20

# ---- Phase 1: frozen encoder ----
print("=" * 60)
print(f"Full Model -- Phase 1: frozen encoder (lr=1e-3, {FULL_EPOCHS_P1} epochs)")
print("=" * 60)

full_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=combined_loss,
    metrics=[dice_coefficient, pupil_iou, iris_iou],
)

full_cbs_p1 = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_dice_coefficient', patience=5,
        mode='max', restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_dice_coefficient', factor=0.5,
        patience=3, mode='max', min_lr=1e-5),
]

full_hist_p1 = full_model.fit(
    train_full_ds,
    validation_data=val_full_ds,
    epochs=FULL_EPOCHS_P1,
    callbacks=full_cbs_p1,
)

# ---- Phase 2: fine-tune entire model ----
print("\n" + "=" * 60)
print(f"Full Model -- Phase 2: fine-tune all (lr=1e-4, {FULL_EPOCHS_P2} epochs)")
print("=" * 60)

full_encoder.trainable = True

full_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=combined_loss,
    metrics=[dice_coefficient, pupil_iou, iris_iou],
)

full_cbs_p2 = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_dice_coefficient', patience=7,
        mode='max', restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_dice_coefficient', factor=0.5,
        patience=3, mode='max', min_lr=1e-6),
    tf.keras.callbacks.ModelCheckpoint(
        str(MODELS_DIR / 'full_best.keras'),
        monitor='val_dice_coefficient',
        mode='max', save_best_only=True),
]

full_hist_p2 = full_model.fit(
    train_full_ds,
    validation_data=val_full_ds,
    epochs=FULL_EPOCHS_P2,
    callbacks=full_cbs_p2,
)

In [ ]:
# ============================================================
# Training visualization
# ============================================================

def _plot_two_phase(h1, h2, metrics, titles, suptitle):
    fig, axes = plt.subplots(1, len(metrics), figsize=(6 * len(metrics), 5))
    if len(metrics) == 1:
        axes = [axes]
    for ax, metric, title in zip(axes, metrics, titles):
        train_v = h1.history.get(metric, []) + h2.history.get(metric, [])
        val_v   = h1.history.get(f'val_{metric}', []) + \
                  h2.history.get(f'val_{metric}', [])
        ep = range(1, len(train_v) + 1)
        ax.plot(ep, train_v, 'b-', label='Train')
        ax.plot(ep, val_v, 'r-', label='Val')
        ax.axvline(x=len(h1.history.get(metric, [])),
                   color='g', ls='--', label='Phase 2 start')
        ax.set_title(title)
        ax.set_xlabel('Epoch')
        ax.legend()
        ax.grid(True, alpha=0.3)
    fig.suptitle(suptitle, fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()

# ROI model curves
_plot_two_phase(
    roi_hist_p1, roi_hist_p2,
    ['loss', 'binary_dice_coeff', 'binary_iou'],
    ['Loss', 'Dice', 'IoU'],
    'Pupil ROI Model Training',
)

# Full model curves
_plot_two_phase(
    full_hist_p1, full_hist_p2,
    ['loss', 'dice_coefficient', 'pupil_iou', 'iris_iou'],
    ['Loss', 'Dice', 'Pupil IoU', 'Iris IoU'],
    'Full Image Model Training',
)

## Evaluation

Compute IoU, Dice, and circle-fitting accuracy for both models on the validation set.

In [ ]:
# ============================================================
# Evaluate both models
# ============================================================

# --- ROI model ---
print("Pupil ROI Model -- Validation")
roi_results = roi_model.evaluate(val_roi_ds)
print(f"  Loss : {roi_results[0]:.4f}")
print(f"  Dice : {roi_results[1]:.4f}")
print(f"  IoU  : {roi_results[2]:.4f}")

# --- Full model ---
print("\nFull Image Model -- Validation")
full_results = full_model.evaluate(val_full_ds)
print(f"  Loss      : {full_results[0]:.4f}")
print(f"  Dice      : {full_results[1]:.4f}")
print(f"  Pupil IoU : {full_results[2]:.4f}")
print(f"  Iris IoU  : {full_results[3]:.4f}")


# --- Circle-fitting accuracy (full model) ---
def fit_circle_from_mask(mask_2d, class_id):
    """Fit a circle to a segmentation mask region via contour moments."""
    binary = (mask_2d == class_id).astype(np.uint8)
    contours, _ = cv2.findContours(
        binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None
    largest = max(contours, key=cv2.contourArea)
    if len(largest) < 5:
        return None
    M = cv2.moments(largest)
    if M['m00'] == 0:
        return None
    cx = M['m10'] / M['m00']
    cy = M['m01'] / M['m00']
    area = cv2.contourArea(largest)
    radius = np.sqrt(area / np.pi)
    return {'x': cx, 'y': cy, 'radius': radius}


n_circle_eval = 20 if QUICK_MODE else 200
print(f"\nCircle-fitting accuracy on {n_circle_eval} validation samples...")
center_errors, radius_errors = [], []
n_eval = min(n_circle_eval, len(X_full_val))
for i in tqdm(range(n_eval), desc="Circle fit"):
    img = X_full_val[i].astype(np.float32) / 255.0
    gt_mask = y_full_val[i]
    pred = full_model.predict(img[np.newaxis, ...], verbose=0)[0]
    pred_mask = np.argmax(pred, axis=-1).astype(np.uint8)

    gt_circle = fit_circle_from_mask(gt_mask, 2)
    pr_circle = fit_circle_from_mask(pred_mask, 2)

    if gt_circle and pr_circle:
        ce = np.sqrt((gt_circle['x'] - pr_circle['x']) ** 2 +
                     (gt_circle['y'] - pr_circle['y']) ** 2)
        re = abs(gt_circle['radius'] - pr_circle['radius'])
        center_errors.append(ce)
        radius_errors.append(re)

if center_errors:
    print(f"  Pupil center error : {np.mean(center_errors):.2f} px "
          f"(median {np.median(center_errors):.2f})")
    print(f"  Pupil radius error : {np.mean(radius_errors):.2f} px "
          f"(median {np.median(radius_errors):.2f})")
else:
    print("  Could not evaluate circle fitting (no valid detections).")

In [ ]:
# ============================================================
# Visualize predictions side-by-side for both models
# ============================================================

def visualize_both_models(roi_model, full_model, n=4):
    fig, axes = plt.subplots(n, 5, figsize=(22, 4.5 * n))
    col_titles = ['Full Input', 'GT (3-class)', 'Full Pred',
                  'ROI Input', 'ROI Pred (pupil)']

    for i in range(n):
        idx_full = np.random.randint(len(X_full_val))
        idx_roi  = np.random.randint(len(X_roi_val))

        # Full model
        img_f = X_full_val[idx_full].astype(np.float32) / 255.0
        gt_f  = y_full_val[idx_full]
        pred_f = full_model.predict(img_f[np.newaxis, ...], verbose=0)[0]
        pred_f_class = np.argmax(pred_f, axis=-1)

        axes[i, 0].imshow(img_f)
        axes[i, 0].set_title(col_titles[0])
        axes[i, 0].axis('off')

        axes[i, 1].imshow(gt_f, cmap='viridis', vmin=0, vmax=2)
        axes[i, 1].set_title(col_titles[1])
        axes[i, 1].axis('off')

        axes[i, 2].imshow(pred_f_class, cmap='viridis', vmin=0, vmax=2)
        axes[i, 2].set_title(col_titles[2])
        axes[i, 2].axis('off')

        # ROI model
        img_r = X_roi_val[idx_roi].astype(np.float32) / 255.0
        pred_r = roi_model.predict(img_r[np.newaxis, ...], verbose=0)[0][..., 0]

        axes[i, 3].imshow(img_r)
        axes[i, 3].set_title(col_titles[3])
        axes[i, 3].axis('off')

        axes[i, 4].imshow(pred_r, cmap='hot', vmin=0, vmax=1)
        axes[i, 4].set_title(col_titles[4])
        axes[i, 4].axis('off')

    plt.tight_layout()
    plt.show()


visualize_both_models(roi_model, full_model)

## Export

1. **ROI model** -- float16 quantized TFLite for in-browser use (`../models/pupil_segnet.tflite`)
2. **Full model** -- TF SavedModel for Cloud Run (`../cloud/model/pupil_segnet/`)

In [ ]:
# ============================================================
# Export ROI model as float16 quantized TFLite
# ============================================================

converter = tf.lite.TFLiteConverter.from_keras_model(roi_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_model = converter.convert()

# Save to project models directory
project_models_dir = Path('..') / 'models'
project_models_dir.mkdir(exist_ok=True)
tflite_path = project_models_dir / 'pupil_segnet.tflite'
with open(tflite_path, 'wb') as f:
    f.write(tflite_model)

size_mb = len(tflite_model) / (1024 * 1024)
print(f"ROI TFLite model: {size_mb:.1f} MB -> {tflite_path.resolve()}")
if size_mb > 2.0:
    print("WARNING: TFLite model exceeds 2 MB target!")
else:
    print("Size is within the <2 MB target.")

In [ ]:
# ============================================================
# Export Full model as SavedModel for Cloud Run
# ============================================================

saved_model_path = Path('..') / 'cloud' / 'model' / 'pupil_segnet'
saved_model_path.parent.mkdir(parents=True, exist_ok=True)

# Keras 3: use model.export() for SavedModel format
full_model.export(str(saved_model_path))

print(f"Full SavedModel exported to: {saved_model_path.resolve()}")

In [ ]:
# ============================================================
# Verify both exports
# ============================================================

print("--- Verifying TFLite (ROI model) ---")
interpreter = tf.lite.Interpreter(model_path=str(tflite_path))
interpreter.allocate_tensors()
inp_det = interpreter.get_input_details()
out_det = interpreter.get_output_details()
print(f"  Input : {inp_det[0]['shape']}, dtype={inp_det[0]['dtype']}")
print(f"  Output: {out_det[0]['shape']}, dtype={out_det[0]['dtype']}")

test_roi = X_roi_val[0:1].astype(np.float32) / 255.0
interpreter.set_tensor(inp_det[0]['index'], test_roi)
interpreter.invoke()
tflite_out = interpreter.get_tensor(out_det[0]['index'])
print(f"  Test output shape: {tflite_out.shape}")
print(f"  Test output range: [{tflite_out.min():.3f}, {tflite_out.max():.3f}]")
print("  TFLite verification: OK")

print("\n--- Verifying SavedModel (Full model) ---")
loaded = tf.saved_model.load(str(saved_model_path))
# Keras 3 export uses 'serve' or 'serving_default' signature
sigs = list(loaded.signatures.keys())
print(f"  Available signatures: {sigs}")
sig_key = 'serving_default' if 'serving_default' in sigs else sigs[0]
infer = loaded.signatures[sig_key]
test_full = tf.constant(X_full_val[0:1].astype(np.float32) / 255.0)
output = infer(test_full)
out_key = list(output.keys())[0]
sm_out = output[out_key].numpy()
print(f"  Input shape : {test_full.shape}")
print(f"  Output shape: {sm_out.shape}")
print(f"  Output key  : {out_key}")
print("  SavedModel verification: OK")

In [ ]:
# ============================================================
# Benchmark inference speed
# ============================================================

N_BENCH = 10 if QUICK_MODE else 50

# --- ROI Keras model ---
test_roi_batch = X_roi_val[0:1].astype(np.float32) / 255.0
times = []
for _ in range(N_BENCH):
    t0 = time.time()
    _ = roi_model.predict(test_roi_batch, verbose=0)
    times.append(time.time() - t0)
print(f"ROI  Keras  : {np.mean(times)*1000:.1f} ms  (avg of {N_BENCH} runs)")

# --- ROI TFLite ---
times = []
for _ in range(N_BENCH):
    t0 = time.time()
    interpreter.set_tensor(inp_det[0]['index'], test_roi_batch)
    interpreter.invoke()
    _ = interpreter.get_tensor(out_det[0]['index'])
    times.append(time.time() - t0)
print(f"ROI  TFLite : {np.mean(times)*1000:.1f} ms  (avg of {N_BENCH} runs)")

# --- Full Keras model ---
test_full_batch = X_full_val[0:1].astype(np.float32) / 255.0
times = []
for _ in range(N_BENCH):
    t0 = time.time()
    _ = full_model.predict(test_full_batch, verbose=0)
    times.append(time.time() - t0)
print(f"Full Keras  : {np.mean(times)*1000:.1f} ms  (avg of {N_BENCH} runs)")

# --- Full SavedModel ---
times = []
for _ in range(N_BENCH):
    t0 = time.time()
    _ = infer(tf.constant(test_full_batch))
    times.append(time.time() - t0)
print(f"Full Saved  : {np.mean(times)*1000:.1f} ms  (avg of {N_BENCH} runs)")

print("\nAll benchmarks complete.")

## Summary

### Two models trained:

| Model | Input | Output | Format | Target |
|-------|-------|--------|--------|--------|
| **Pupil ROI** | 128x128 RGB | 128x128x1 sigmoid | TFLite (float16) | In-browser (TF.js) |
| **Full Image** | 256x256 RGB | 256x256x3 softmax | SavedModel | Cloud Run |

### Exported artifacts:
- `../models/pupil_segnet.tflite` -- ROI model for browser (<2 MB)
- `../cloud/model/pupil_segnet/` -- Full model SavedModel for serving

### Datasets used:
- MOBIUS (up to 3,559 real phone-camera images)
- iBUG Eye Segmentation (up to ~2K images)
- Roboflow pupilX (up to 804 images)
- Enhanced synthetic augmentation (15,000 images)

### Next steps:
1. Place `pupil_segnet.tflite` in the project `models/` directory (already done by export)
2. Deploy the SavedModel to Cloud Run (see `cloud/` directory)
3. Convert TFLite to TF.js format if needed: `tensorflowjs_converter --input_format=tflite ../models/pupil_segnet.tflite ../models/tfjs/`
4. Fine-tune on additional real data as it becomes available